In [5]:
import torch
from pathlib import Path

from lit_lraspp_mobilenet_v3_large import LRASPPMobileNetV3Large
def train():
    pass

DEVICE = torch.device('cpu')


In [6]:
weights_path = "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/mussels/LRASPP/best-val_miou=0.8405-epoch=27-step=3387.ckpt"
torchscript_path = "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/mussels/LRASPP/LRASPP_MobileNetV3_mussel_presence_jit.pt"
model = LRASPPMobileNetV3Large.load_from_checkpoint(weights_path, train=False, map_location=DEVICE, strict=False)
# model.to_torchscript(file_path=torchscript_path, method='trace', example_inputs=x)

In [7]:
# Save state_dict only
torch.save(model.state_dict(), Path(weights_path).with_suffix(".pt"))

In [8]:

x = torch.rand(1, 3, 8, 8, device=DEVICE, requires_grad=False)

model.eval()
model.requires_grad_(False)
with torch.no_grad():
    traced_model = torch.jit.trace(model, example_inputs=x)
    traced_model.save(torchscript_path)
# torch.onnx.export(model, x, "torchscript_files/LRASPP_MobileNetV3_kelp_species.onnx",
#                   opset_version=11, verbose=True)

In [33]:
# torch.onnx.export(model, x, onnx_path,
#                   input_names=['input'],
#                   output_names=['output'],
#                   dynamic_axes={
#                       'input': {
#                           0: 'batch_size',
#                           2: 'width',
#                           3: 'length'
#                       },
#                       'output': {
#                           0: 'batch_size',
#                           2: 'width',
#                           3: 'length'
#                       }
#                   })

NameError: name 'onnx_path' is not defined

In [9]:
# % pip install onnx onnxruntime-gpu

Note: you may need to restart the kernel to use updated packages.


In [10]:
# import onnx
#
# onnx_model = onnx.load(onnx_path)
# onnx.checker.check_model(onnx_model)

In [11]:
# import onnxruntime as ort
# import numpy as np
#
# z = np.ones((1, 3, 128, 128)).astype(np.float32)
# print(z.dtype)
# ort_sess = ort.InferenceSession(onnx_path,
#                                 providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider',
#                                            'CPUExecutionProvider'])
# outputs = ort_sess.run(None, {'input': z})

float32


In [50]:
print(np.argmax(np.asarray(outputs[0]), axis=1))

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [51]:
z - np.expand_dims(np.array([0.1, 0.2, 0.3]), axis=[1, 2])

array([[[[0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         ...,
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9]],

        [[0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         ...,
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8]],

        [[0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         ...,
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7]]]])

In [61]:
from torch import package

path = '/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/mussels/lraspp_mussels.pt'
package_name = 'lraspp_mussels'
resource_name = "model.pkl"

with package.PackageExporter(path) as exp:
    exp.intern(["lit_lraspp_mobilenet_v3_large", "utils.**", "kelp_data_module",
                "geotiff_crop_dataset.**"])
    exp.extern(
        ["pytorch_lightning.**", "torchmetrics.**", "torchvision.**", "numpy", "PIL.**", "tqdm.**",
         "rasterio"])
    exp.save_pickle(package_name, resource_name, model)